In [ ]:
import pandas as pd
import numpy

In [ ]:
df_specials = pd.read_excel(r"COMBINED SPECIALS FILE.xlsx")
df_master = pd.read_csv(r"MASTER DICTIONARY.csv")
df_data = pd.read_csv(r"CLEANED RAW DATA.csv")
df_protocols = pd.read_excel(r"CLEANED PROTOCOL DATA.xlsx")

In [ ]:
row_count_data = sum(1 for row in df_data["Invoice #"])
row_count_specials = sum(1 for row in df_specials["CPT"])
row_count_master = sum(1 for row in df_master["Description"])
row_count_protocol = sum(1 for row in df_protocols["Employer"])

In [ ]:
df_data["Employer Name"].fillna(0,inplace = True)
df_data["Dept"].fillna(0,inplace = True)
df_data["CPT"].fillna(0,inplace = True)

In [ ]:
def conversion(num):
    if "(" in num and "," not in num:
        val = num.split("(")
        val = val[1].split(")")
        val = val[0].split("$")
        val = float(val[1])
        return val
    elif "(" in num and "," in num:
        val = num.split("(")
        val = val[1].split(")")
        val = val[0].split("$")
        val = val[1].split(",")
        val = ''.join(val)
        val = float(val)
        return val
    elif "(" not in num and "," not in num and "$" in num:
        val = num.split("$")
        val = float(val[1])
        return val
    elif "(" not in num and "," in num:
        val = num.split("$")
        val = val[1].split(",")
        val = ''.join(val)
        val = float(val)
        return val
    elif "N/A" in num:
        val = "N/A"
        return val

In [ ]:
protocol_dict = {}

for i in range(0, row_count_protocol):
    emp_name = df_protocols["Employer"][i].lower()
    cpt1 = df_protocols["CPT Code"][i]
    if type(cpt1) == str:
        cpt = cpt1.lower()
    else:
        cpt  = cpt1
        
    if emp_name not in protocol_dict:
        protocol_dict[emp_name] = [cpt]
    else:
        protocol_dict[emp_name].append(cpt)

In [ ]:
specials_dict = {}
for i in range(0,row_count_specials):
    empname1 = str(df_specials["Employer"][i])
    empname = empname1.lower()
    
    depname1 = str(df_specials["Department"][i])
    depname2 = depname1.lower()
    if "attn" in depname2:
        depname = depname2
    else:
        depname = "attn: "+depname2

    concatempdep = empname+" "+depname
    
    if concatempdep not in specials_dict:
                
        cpt1 = str(df_specials["CPT"][i])
        cpt = cpt1.lower()
        
        occ_med_fee1 = df_specials["Occ Med Fee"][i]
        if type(occ_med_fee1) == str:
            occ_med_fee = conversion(occ_med_fee1)
        else:
            occ_med_fee = occ_med_fee1
            
        cpt_dict = {}
        cpt_dict[cpt] = occ_med_fee
        specials_dict[concatempdep] = cpt_dict
    
    else:
        cpt1 = str(df_specials["CPT"][i])
        cpt = cpt1.lower()
        
        cpt_dict = specials_dict[concatempdep]
        
        if cpt not in cpt_dict:
            occ_med_fee1 = df_specials["Occ Med Fee"][i]
            if type(occ_med_fee1) == str:
                occ_med_fee = conversion(occ_med_fee1)
            else:
                occ_med_fee = occ_med_fee1

            specials_dict[concatempdep][cpt] = occ_med_fee

In [ ]:
master_dict = {}
for i in range(0,row_count_master):
    cpt1 = str(df_master["CPT Code"][i])
    cpt = cpt1.lower()
    
    if cpt not in master_dict:
        cost1 = df_master["Region 1 - WNY"][i]
        if type(cost1) == str:
            cost = conversion(cost1)
        else:
            cost = cost1
        master_dict[cpt] = cost


In [ ]:
correct_charge_list = []
spl_or_master = []

for i in range(0,row_count_data):
    empname1 = df_data["Employer Name"][i]
    empname = empname1.lower()
    
    depname1 = str(df_data["Dept"][i])
    depname2 = depname1.lower()
    if "attn" in depname2:
        depname = depname2
    else:
        depname = "attn: "+depname2

    concatempdep = empname+" "+depname
    
    cpt1 = df_data["CPT"][i]
    if type(cpt1) == str:
        cpt = cpt1.lower()
    else:
        cpt = cpt1
        
    if concatempdep in specials_dict:
#         cptn1 = df_data["CPT"][i]
#         if type(cptn1) == str:
#             cptn = cptn1.lower()
#         else:
#             cptn = cptn1
        
        cpt_dict = specials_dict[concatempdep]
        if cpt in cpt_dict:
            charge = specials_dict[concatempdep][cpt]
            spl_or_master.append("Special file")
        elif cpt in master_dict:
            charge = master_dict[cpt]
            spl_or_master.append("Master file")
        else:
            charge = "N/A"
            spl_or_master.append(" ")
    
    elif cpt in master_dict:
        charge = master_dict[cpt]
        spl_or_master.append("Master file")
    
    elif concatempdep == 0 and cpt == 0:
        charge = 0
        spl_or_master.append(" ")
    
    else:
        charge = "N/A"
        spl_or_master.append(" ")
        
    correct_charge_list.append(charge)

In [ ]:
variance_list = []

for i in range(0,row_count_data):
    if correct_charge_list[i] == 0.0:
        variance_list.append(0.0)
    elif correct_charge_list[i] == "N/A":
        variance_list.append("N/A")
    else:
        if type(df_data["Balance"][i]) == str:
            bal = conversion(df_data["Balance"][i])
        else:
            bal = float(df_data["Balance"][i])
        
        if type(correct_charge_list[i]) == str:
            ccl = conversion(str(correct_charge_list[i]))
        elif correct_charge_list[i] is None:
            ccl = float(0)
        else:
            ccl = float(correct_charge_list[i])
        if type(bal) == str or type(ccl) == str:
            variance_list.append("N/A")
        else:
            calc = ccl - bal
            variance_list.append(calc)

In [ ]:
#testing
if correct_charge_list[158] == 0.0:
    print("first if",0)
elif correct_charge_list[158] == "N/A":
    print("elif - N/A")
else:
    if type(df_data["Balance"][158]) == str:
        bal = conversion(df_data["Balance"][158])
    else:
        bal = df_data["Balance"][158]
    
    if type(ccl) == str:
        ccl = conversion(correct_charge_list[158])
    else:
        ccl = correct_charge_list[158]
        
    if type(bal) == str or type(ccl) == str:
        print("else: if  - N/A")
    else:
        calc = ccl - bal
        print("last else else",calc)
            

In [ ]:
wrong_cpt_based_on_protocols = []

for i in range(0,row_count_data):
    empname1 = str(df_data["Employer Name"][i])
    empname = empname1.lower()
    
    cpt1 = df_data["CPT"][i]
    if type(cpt1) == str:
        cpt = cpt1.lower()
    else:
        cpt = cpt1
    
    if empname in protocol_dict:
        dict_cpt = protocol_dict[empname]
        if cpt not in dict_cpt:
            wrong_cpt_based_on_protocols.append("Wrong CPT based on protocols")
        else:
            wrong_cpt_based_on_protocols.append(" ")
    else:
        wrong_cpt_based_on_protocols.append(" ")

# Copy the 3 columns from output file to the main file in the data sheet

In [ ]:
df_data["Correct_Charge"] = correct_charge_list
df_data["Variance"] = variance_list
df_data["Wrong CPT based on protocols"] = wrong_cpt_based_on_protocols
df_data["CPT comes from : "] = spl_or_master


op_path = r"OUTPUT FILE.xlsx"
df = pd.DataFrame(list(zip(correct_charge_list, variance_list, wrong_cpt_based_on_protocols, spl_or_master)), columns = ['Correct Charge', 'Variance','Wrong CPT based on protocols',"CPT comes from : "])
df.to_excel(op_path,index = False,sheet_name = 'data')